## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-17-08-47-22 +0000,wapo,"Trump, buoyed by Gaza deal, plans Putin summit...",https://www.washingtonpost.com/politics/2025/1...
1,2025-10-17-08-40-36 +0000,nypost,Massachusetts school staffer dies after she’s ...,https://nypost.com/2025/10/17/us-news/massachu...
2,2025-10-17-08-37-15 +0000,bbc,Major UK rare earths refinery scrapped in favo...,https://www.bbc.com/news/articles/czxnv7y03nno...
3,2025-10-17-08-36-00 +0000,wsj,"Global Markets Largely Down, Gold Extends Stre...",https://www.wsj.com/finance/stocks/global-mark...
4,2025-10-17-08-30-24 +0000,nyt,"Tomiichi Murayama, Japanese Leader Who Gave Wa...",https://www.nytimes.com/2025/10/17/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2444/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,64
178,new,21
84,will,17
133,china,15
398,he,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
169,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,143
230,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...,128
119,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,125
285,2025-10-16-14-49-58 +0000,nypost,Trump says CBS under owner David Ellison will ...,https://nypost.com/2025/10/16/media/trump-says...,124
11,2025-10-17-08-00-00 +0000,cbc,Why Ukraine-Russia peace will be harder for Tr...,https://www.cbc.ca/news/world/trump-ukraine-ru...,119


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
169,143,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
51,59,2025-10-17-03-46-08 +0000,nypost,"Zohran Mamdani dodges attacks, straight answer...",https://nypost.com/2025/10/16/us-news/mamdani-...
265,55,2025-10-16-15-47-55 +0000,cbc,"Trump's Gaza deal may be 'historic,' but falls...",https://www.cbc.ca/news/world/gaza-trump-peace...
127,47,2025-10-16-22-31-31 +0000,startribune,"Prison for woman who admitted she was driving,...",https://www.startribune.com/prison-for-woman-a...
181,44,2025-10-16-20-21-13 +0000,nypost,Thune says ‘time’s right’ for Russia sanctions...,https://nypost.com/2025/10/16/us-news/thune-sa...
298,43,2025-10-16-14-00-14 +0000,nypost,"Chaos erupts after Kenyan forces fire in air, ...",https://nypost.com/2025/10/16/world-news/kenya...
229,41,2025-10-16-17-42-00 +0000,wsj,Weaker Oil Prices Threaten European Oil Majors...,https://www.wsj.com/business/energy-oil/weaker...
139,40,2025-10-16-21-54-57 +0000,cbc,"John Bolton, Trump's ex-national security advi...",https://www.cbc.ca/news/world/john-bolton-char...
259,38,2025-10-16-16-00-00 +0000,wsj,An extended government shutdown raises the pro...,https://www.wsj.com/economy/central-banking/fe...
296,36,2025-10-16-14-02-46 +0000,wapo,Do you need a better job? Answer these five qu...,https://www.washingtonpost.com/business/2025/1...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
